# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Introducción al aprendizaje automático

### Trabajo práctico entregable - Grupo 22 - Parte 2
Armado de un esquema de aprendizaje automático

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno
import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

from sklearn.tree import export_graphviz
from graphviz import Source


## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [ ]:
dataset = pd.read_csv("./data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

msno.bar(dataset)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Antes de responder a las preguntas, veamos la descripción del dataset.

## Loan dataset based on the Kaggle Home Equity dataset	
Available at: https://www.kaggle.com/ajay1735/hmeq-data	
	
## Context	
## -----------------------------------------------

The consumer credit department of a bank wants to automate the decisionmaking process for approval of home equity lines of credit. To do this, they will follow the recommendations of the Equal Credit Opportunity Act to create an empirically derived and statistically sound credit scoring model. The model will be based on data collected from recent applicants granted credit through the current process of loan underwriting. The model will be built from predictive modeling tools, but the created model must be sufficiently interpretable to provide a reason for any adverse actions (rejections).	

## Content	
## -----------------------------------------------

The Home Equity dataset (HMEQ) contains baseline and loan performance information for 5960 recent home equity loans. The target (BAD) is a binary variable indicating whether an applicant eventually defaulted or was seriously delinquent. This adverse outcome occurred in 1189 cases (20%). For each applicant, 12 input variables were recorded.

## Attributes	
## -----------------------------------------------

* Name:    Description.	
* TARGET:  Label. 1 = client defaulted on loan; - 0 = loan repaid.	
* LOAN:    Amount of the loan request.
* MORTDUE: Amount due on existing mortgage.	
* VALUE:   Value of current property.
* YOJ:     Years at present job.
* DEROG:   Number of major derogatory reports.	
* DELINQ:  Number of delinquent credit lines.
* CLAGE:   Age of oldest trade line in months.
* NINQ:    Number of recent credit lines.
* CLNO:    Number of credit lines.
* DEBTINC: Debt-to-income ratio.


# Ahora respondemos a las preguntas.

1. *¿De qué se trata el conjunto de datos?* El dataset proviene de Kaggle, contiene datos financieros de 5960 préstamos. La idea del departamento de créditos del banco es automatizar el proceso de toma de decisiones entrenando un modelo con este dataset.


2. *¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?* La variable objetivo que hay que predecir es **Target**, la cual indica si se le va a otorgar un crédito a la persona solicitante o no.


3. *¿Qué información (atributos) hay disponible para hacer la predicción?* Las variables disponibles del dataset para hacer la predicción son:

* **LOAN**:    Variable numérica. Cantidad del préstamo.
* **MORTDUE**: Variable numérica. Lo que ya debe la persona?.	
* **VALUE**:   Variable numérica. Valor de la propiedad.
* **YOJ**:     Variable numérica. Años en el presente trabajo.
* **DEROG**:   Variable numérica. Número de reportes negativos.	
* **DELINQ**:  Variable numérica. Número de líneas crediticias delinquivas?.
* **CLAGE**:   Variable numérica. Edad de la 'trade line' (sea lo que sea esto) en meses.
* **NINQ**:    Variable numérica. Número de líneas de crédito recientes.
* **CLNO**:    Variable numérica. Número de líneas de crédito (totales?).
* **DEBTINC**: Variable numérica. tasa deuda-ingreso.


4. *¿Qué atributos imagina ud. que son los más determinantes para la predicción?* En orden de importancia: 

- **Loan**: El tamaño del préstamo tiene que ser relevante.
- **DEBTINC**: Esta variable indica directamente la cantidad real de ingresos que una persona dispone. 
- **MORTDUE**: Lo que debe la persona claramente tiene que ser relevante.\
- **YOJ**: Si bien no creo que sea tan determinante, mucho tiempo en el mismo trabajo al menos implica un flujo constante de dinero.
- **DELINQ**: Esta variable nos dice directamente cuántos créditos no pudo pagar. Tiene que ser importante.

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

## Ejercicio 3: Árboles de Decisión


### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Sin embargo, es importante evaluar estadisticamente el comportamiento de nuestro modelo. Una posibilidad es comparar los valores predichos por el árbol con los valores observados para un conjunto relativamente grande de casos. Este conjunto es el que denominamos conjunto de "testing". 
Podemos utilizar una tabla o **matriz de contingencia** para calcular las frecuencias relativas de los eventos. En la literatura de machine learning esta matriz se la llama **matriz de confusión** (confusion matrix).

Los elementos de la matriz son:

* TP - verdaderos positivos: se pronosticó el pago del cŕedito y ocurrió (aciertos, hits)
* TN - verdaderos negativos: se pronosticó el default y ocurrió (aciertos negativos)
* FN - falsos negativos o sorpresas: se pronosticó el pago del cŕedito y ocurrió un default (misses)
* FP - falsos positivos o falsa alarma: se pronosticó un default pero se pagó el crédito (false alarm)

Existen multiples índices (scores) que pueden calcularse a partir de la matriz de contingencia:

$$\text{Precision}=\frac{TP}{TP+FP}$$

es la porción de eventos pronosticados que resultó en una correcta detección. El valor va entre 0 y 1, siendo este último el valor óptimo.

$$\text{Accuracy} =\frac{TP+TN}{TP+TN+FP+FN}$$

es la fracción de eventos pronosticados correctamente sobre todas las predicciones.

$$\text{Recall} = \frac{TP}{FN+TP}$$

es la fracción de eventos positivos observados que fueron correctamente pronosticados. El valor va entre 0 y 1, siendo este último el valor óptimo.

$$ \text{F1} =\frac{ precision * recall} {precision + recall} $$

es una media armónica de la precisión y el recall. Toma valores entre 0 y 1, siendo 1 el mejor valor y 0 el peor.

$$\text{false alarm ratio} = \frac{FP}{TP+FP}$$

es la porción de eventos pronosticados que resultó en una falsa detección. El valor va entre 0 y 1, siendo el primero el valor óptimo.

$$\text{bias} = \frac{TP+FP}{TP+FN}$$

es la relación entre la frecuencia con la que se pronostica el evento y la frecuencia con la que el evento ocurre. Su valor está entre menos infinito e infinito. El valor óptimo es 1.

$$\text{Critical success index} = \frac{TP}{TP+FP+FN}$$

El ETS es similar al CSI pero remueve el efecto de los aciertos que pueden ocurrir por azar. Su valor máximo es 1, que también es el valor óptimo.

Notar que estos índices se orientan hacia la categoría de más interés (en este caso, que se haya pagado el crédito). Lo que se mide son aciertos en el pago del crédito o falsas detecciones del pago.

In [ ]:
def compute_scores_class( y_pred , y_true ) :
    
    "defino una funcion para calcular la matriz de contingencia, y varios scores"
    
    cmatrix=sklearn.metrics.cluster.contingency_matrix( y_true , y_pred )

    cmatrix_norm = cmatrix/np.sum(cmatrix)
        
    tp = cmatrix_norm[0,0]
    fp = cmatrix_norm[1,0]
    tn = cmatrix_norm[1,1]
    fn = cmatrix_norm[0,1]
    n  = np.sum( cmatrix_norm ) #Esto deberia dar siempre 1 si la matriz esta normalizada. 
    
    model_pre = tp / (tp+fp)
    model_acc = (tp+tn)/(tp+tn+fp+fn)
    model_rec = tp / ( fn + tp )
    model_far = fp / ( tp + fp )
    model_bias= ( tp + fp ) / ( tp + fn )
    model_csi = tp / ( tp + fp + fn )
    model_random_hits = ((tp+fp)*(tp+fn))/n
    model_ets = ( tp - model_random_hits ) / ( tp + fp + fn - model_random_hits )
    F1 = 2 * (model_pre * model_rec) / (model_pre + model_rec)
    
    index = {"FAR":model_far,"BIAS":model_bias,"F1":F1,
               "CSI":model_csi,"RND":model_random_hits,"ETS":model_ets,
               "PREC":model_pre,"ACC":model_acc,"REC":model_rec}
    
    return cmatrix_norm, index


Definimos nuestros conjuntos de entrenamiento y evaluación con las variables predictoras que quiero usar. Corroboro que en los conjuntos de entrenamiento y evaluación, la relación de etiquetas 'default' y 'repago' se mantengan respecto del dataset total.

In [ ]:

#["LOAN","MORTDUE","VALUE","YOJ","DEROG","DELINQ","CLAGE","NINQ","CLNO","DEBTINC"]

predictores = ["LOAN","VALUE","DEBTINC"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# los datos deben estar en arrays
x_values = np.array([dataset[x] for x in predictores], dtype=object).transpose()
y_values = dataset["TARGET"]  #variable a predecir

# separamos los datos en entrenamiento (train) y testeo (testeo)
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, random_state = 78014,
                                                    test_size = 0.2, shuffle=True)

#Corroboro que están bien distribuidos los datos
loans_d_tot = len(dataset[dataset["TARGET"]==1])
loans_r_tot = len(dataset[dataset["TARGET"]==0])
print("default/repaid total ratio:",loans_d_tot/loans_r_tot)

loans_d_train = len(y_train[y_train==1])
loans_r_train = len(y_train[y_train==0])
print("default/repaid train ratio:",loans_d_train/loans_r_train)

loans_d_test = len(y_test[y_test==1])
loans_r_test = len(y_test[y_test==0])
print("default/repaid test ratio:",loans_d_test/loans_r_test)


Entreno ahora un árbol de decisión con profundidad 2, que es la profundidad mínima donde se pronostica que se repagan algunos créditos:

In [ ]:
#Entreno el arbol
depth=3
tree_loan = DecisionTreeClassifier(max_depth = depth, splitter='best')
tree_loan.fit(x_train, y_train)
y_pred = tree_loan.predict(x_test)

tree.plot_tree(tree_loan, 
               feature_names = predictores,
               class_names = ["d","r"],   #d=default, #r=repaid
               rounded=True, 
               filled=True)

Evaluemos ahora qué tan bien se comporta nuestro modelo. 

Para eso, primero veamos la matriz de contingencia. Observamos que todos los aciertos del modelo está al pronosticar que la gente pagó su crédito, pero no detectó ningún caso de gente que terminó en default.

Esto se puede analizar mejor con los índices antes definidos:
* La exactitud, precisión y índice de acierto rítico son básicamente los mismos ya que el modelo predice sólo una clase.
* El recall es 1 por la misma razón, no hay verdaderos negativos.
* Qué podemos decir sobre el F1??? 
* El modelo es propenso a sobreestimar el default: 15% de los casos, viendo la tasa de falsas alarmas. El bias > 1 también indica este fenómeno.

In [ ]:
cmatrix, index = compute_scores_class( y_pred , y_test )

res = sns.heatmap(cmatrix , 
                  annot=True, 
                  fmt='.2f', 
                  cmap="YlGnBu", 
                  vmin=0.0, 
                  vmax=1.0 , 
                  xticklabels=[0,1],
                  yticklabels=[0,1])

print("precisión: ", index["PREC"])
print("exactitud: ", index["ACC"])
print("Recall: ", index["REC"])
print("F1: ",index["F1"])
print("tasa de falsas alarmas: ", index["FAR"])
print("bias: ", index["BIAS"])
print("índice de acierto crítico: ", index["CSI"])

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
from sklearn.model_selection import GridSearchCV

criterion = ["gini", "entropy", "log_loss"]
max_depth = [3,4,5,6,7,8,9,10]
min_samples_leaf = [1,2,3,4,5]

#defino el modelo
tree_loan = DecisionTreeClassifier()

#defino el diccionario de parámetros para armar la grilla
params = {'criterion': criterion,'max_depth': max_depth,'min_samples_leaf': min_samples_leaf}

#Instancio el objeto de búsqueda
trees = GridSearchCV(
        estimator=tree_loan,   #mi estimador 
        param_grid=params,     #la grilla donde voy a variar los hiperparámetros
        cv=None,               #None to use 5-fold cross-validation
        n_jobs=4,              #4 búsquedas en simultáneo
        verbose=1          
        )

#realizo la búsqueda
trees.fit(x_train, y_train)
#mejores parámetros:
print(trees.cv_results_)


print(y_pred)


# tree.plot_tree(tree_loan, 
#                feature_names = predictores,
#                class_names = ["d","r"],   #d=default, #r=repaid
#                rounded=True, 
#                filled=True)